In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('성별_전체_추계인구.csv', encoding='euc-kr')

In [3]:
# 80세이상 - 를 0으로 변환
df.replace('-', 0, inplace=True)

In [4]:
# 모든 숫자 열을 정수형으로 변환
for col in df.columns[2:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [5]:
# '계' 데이터를 필터링하여 별도로 저장
total_population = df[df['연령별'] == '계']

# 유소년 인구 데이터 필터링 (0-14세)
youth_data = df[df['연령별'].str.match('^(0 - 4세|5 - 9세|10 - 14세)$')]

# 65세 미만 인구 데이터 필터링 (15-64세)
under_65_data = df[df['연령별'].str.match('^(15 - 19세|20 - 24세|25 - 29세|30 - 34세|35 - 39세|40 - 44세|45 - 49세|50 - 54세|55 - 59세|60 - 64세)$')]

# 65세 이상 인구 데이터 필터링 (65세 이상)
over_65_data = df[df['연령별'].str.match('^(65 - 69세|70 - 74세|75 - 79세|80 - 84세|85 - 89세|90 - 94세|95 - 99세|100세 이상)$')]


In [6]:
# 유소년 인구 합산
youth_population = youth_data.drop(columns=['성별']).set_index('연령별').sum().to_frame().T
youth_population['성별'] = '전체'
youth_population['연령별'] = '유소년'

# 65세 미만 인구 합산 (노년x, 즉 15-64세)
under_65_population = under_65_data.drop(columns=['성별']).set_index('연령별').sum().to_frame().T
under_65_population['성별'] = '전체'
under_65_population['연령별'] = '생산연령'

# 65세 이상 인구 합산
over_65_population = over_65_data.drop(columns=['성별']).set_index('연령별').sum().to_frame().T
over_65_population['성별'] = '전체'
over_65_population['연령별'] = '노년'

In [7]:
# '계' 데이터와 합산된 데이터를 결합
combined_population = pd.concat([total_population, youth_population, under_65_population, over_65_population])

In [8]:
print(combined_population)

   성별  연령별      1960      1961      1962      1963      1964      1965  \
0  전체    계  25012374  25765673  26513030  27261747  27984155  28704674   
0  전체  유소년  10587583  11065019  11472623  11850374  12215638  12577740   
0  전체  노년x  13698341  13949773  14257691  14595205  14923382  15245851   
0  전체   노년    667268    685487    714249    744047    769547    801189   

       1966      1967  ...      2014      2015      2016      2017      2018  \
0  29435571  30130983  ...  50746659  51014947  51217803  51361911  51585058   
0  12916059  13199493  ...   7213693   7029883   6864563   6724283   6595054   
0  15592878  16003566  ...  37255840  37443896  37596157  37571568  37623919   
0    843300    842163  ...   6277126   6541168   6757083   7066060   7366085   

       2019      2020      2021      2022      2023  
0  51764822  51836239  51769539  51672569  51712619  
0   6448080   6305870   6148341   5947964   5705235  
0  37627748  37378502  37051333  36743472  36571568  
0   7688994 

In [9]:
# 결과를 CSV 파일로 저장
combined_population.to_csv('성별_전체_추계인구_유소년노년.csv', index=False, encoding='euc-kr')